In [5]:
import lucem_illud #pip install -U git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git

#All these packages need to be installed from pip
#For NLP
import nltk
import re
import numpy as np #For arrays
import pandas #Gives us DataFrames
import matplotlib.pyplot as plt #For graphics
import seaborn #Makes the graphics look nicer

#Displays the graphs
import graphviz #You also need to install the command line graphviz

#These are from the standard library
import os.path
import zipfile
import subprocess
import io
import tempfile
import lucem_illud.stanford as stanford

%matplotlib inline

//anaconda/lib/python3.5/site-packages/nltk/tag/stanford.py:183: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordNERTagger, self).__init__(*args, **kwargs)
//anaconda/lib/python3.5/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


## <span style="color:red">*Exercise 4*</span>

<span style="color:red">How would you extract relevant information about the Trayvon Martin sentence directly from the dependency parse (above)? Code an example here. (For instance, what compound nouns show up with what verb phrases within the sentence?) How could these approaches inform your research project?

In [6]:
text = ['I saw the elephant in my pajamas.', 
        'The quick brown fox jumped over the lazy dog.', 
        'While in France, Christine Lagarde discussed short-term stimulus efforts in a recent interview with the Wall Street Journal.', 
        'Trayvon Benjamin Martin was an African American from Miami Gardens, Florida, who, at 17 years old, was fatally shot by George Zimmerman, a neighborhood watch volunteer, in Sanford, Florida.', 
        'Buffalo buffalo Buffalo buffalo buffalo buffalo Buffalo buffalo']

ieDF = stanford.openIE(text)
ieDF

Starting OpenIE run
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[main] INFO edu.stanford.nlp.tagger.maxent.MaxentTagger - Loading POS tagger from edu/stanford/nlp/models/pos-tagger/english-left3words/english-left3words-distsim.tagger ... done [1.3 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator depparse
[main] INFO edu.stanford.nlp.parser.nndep.DependencyParser - Loading depparse model file: edu/stanford/nlp/models/parser/nndep/english_UD.gz ... 
[main] INFO edu.stanford.nlp.parser.nndep.Classifier - PreComputed 99996, Elapsed Time: 15.613 (s)
[main] INFO edu.stanford.nlp.parser.nndep.DependencyParser - Initializing dependency parser ... done [17.2 sec].
[main] INFO edu.stanford.nlp.pip

,certainty,subject,verb,object
0,1.0,elephant,is in,my pajamas
1,1.0,I,saw,elephant in my pajamas
2,1.0,I,saw,elephant
3,1.0,quick brown fox,jumped over,lazy dog
4,1.0,quick brown fox,jumped over,dog
5,1.0,quick fox,jumped over,dog
6,1.0,fox,jumped over,dog
7,1.0,brown fox,jumped over,lazy dog
8,1.0,brown fox,jumped over,dog
9,1.0,quick fox,jumped over,lazy dog


In [7]:
ieDF[ieDF['subject'] == 'Trayvon Benjamin Martin']['verb']

25    was African American from
26            was American from
27                          was
28                          was
Name: verb, dtype: object

In [8]:
ieDF[ieDF['subject'] == 'Trayvon Benjamin Martin']['object']

25             Florida
26             Florida
27            American
28    African American
Name: object, dtype: object

<span style="color:blue"> I may use this method to find the usage of concepts in philosophical work. For instance, the verb and object can give me information about what a concept means in a specific paragraph.

## <span style="color:red">*Exercise 5*</span>

<span style="color:red">In the cells immediately following, perform open information extraction on a modest subset of texts relevant to your final project. Analyze the relative attachment of several subjects relative to verbs and objects and visa versa. Describe how you would select among these statements to create a database of high-value statements for your project and then do it by extracting relevant statements into a pandas dataframe.

In [12]:
targetDir = 'sentence'
platoText = []
platoFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding = 'latin-1') as f:
        platoText.append(f.read())
    platoFileName.append(file.name)

paragraph = pandas.DataFrame({'text' : platoText})
remove = np.array([r'\[\d+\]', r'\(\d+\)', '\n', '\t', '\d+', r'\(\w+\)'])
for i in range(paragraph.shape[0]):
    for j in range(len(remove)):
        paragraph["text"].iloc[i] = re.sub(remove[j], ' ', paragraph["text"].iloc[i])
paragraph

,text
0,In reality I had set my heart at that time on ...
1,The method of this man is quite contrary to th...
2,Which of them has been provisionally victoriou...
3,This is simply the long history of the origin ...
4,"Entertaining, as I do, these thoughts, I am, l..."
5,"Undoubtedly the bad conscience is an illness, ..."
6,What is the meaning of ascetic ideals? In arti...
7,"And now, after we have caught sight of the _as..."
8,When he has to tackle sufferers of the lower o...


In [13]:
ieDF = stanford.openIE(paragraph['text'][1])
ieDF

Starting OpenIE run
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[main] INFO edu.stanford.nlp.tagger.maxent.MaxentTagger - Loading POS tagger from edu/stanford/nlp/models/pos-tagger/english-left3words/english-left3words-distsim.tagger ... done [1.2 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator depparse
[main] INFO edu.stanford.nlp.parser.nndep.DependencyParser - Loading depparse model file: edu/stanford/nlp/models/parser/nndep/english_UD.gz ... 
[main] INFO edu.stanford.nlp.parser.nndep.Classifier - PreComputed 99996, Elapsed Time: 13.354 (s)
[main] INFO edu.stanford.nlp.parser.nndep.DependencyParser - Initializing dependency parser ... done [14.8 sec].
[main] INFO edu.stanford.nlp.pip

,certainty,subject,verb,object
0,1.000000,material,creates concept for,himself
1,1.000000,material,creates,concept of bad
2,1.000000,material,then creates concept for,himself
3,1.000000,material,creates concept of bad for,himself
4,1.000000,method,is contrary to,that
5,1.000000,method,is contrary to,quite that
6,1.000000,material,creates,concept
7,1.000000,material,then creates concept of bad for,himself
8,1.000000,material,then creates,concept
9,1.000000,material,then creates,concept of bad


In [14]:
ieDF['subject'].value_counts()

Our audacity             12
he                       10
tools                     8
man                       8
it                        8
material                  8
tame man                  8
Europe                    7
I                         6
Pericles                  6
truth                     5
aristocratic races        4
civilisation              4
races                     4
poets                     4
nay                       4
core                      3
hidden core               3
method                    2
idea                      2
pride                     2
One                       2
wild student 's prank     2
they                      2
student 's prank          2
nonchalance               2
reference                 1
worm man                  1
question                  1
wild student              1
identical contrary        1
bearers                   1
vindictive instincts      1
essential act             1
one                       1
pre-Aryan population

In [17]:
ieDF[ieDF['subject'] == 'truth']['object'].value_counts()

animal                 2
prey                   1
domesticated animal    1
tame animal            1
Name: object, dtype: int64

In [21]:
ieDF[ieDF['subject'] == 'tame man']['verb'].value_counts()

consider    7
learnt      1
Name: verb, dtype: int64

In [19]:
ieDF[ieDF['subject'] == 'tame man']['object'].value_counts()

inner meaning         1
himself               1
man                   1
principle             1
goal                  1
consider              1
historic principle    1
meaning               1
Name: object, dtype: int64

In [20]:
ieDF[ieDF['subject'] == 'races']['object'].value_counts()

their ideals        2
degraded            1
finally degraded    1
Name: object, dtype: int64

In [22]:
ieDF[ieDF['subject'] == 'tools']['object'].value_counts()

more of argument against civilisation more    1
more of argument                              1
more of argument more                         1
more more                                     1
disgrace                                      1
more of argument against civilisation         1
more                                          1
humanity                                      1
Name: object, dtype: int64

In [23]:
ieDF[ieDF['subject'] == 'tools']['verb'].value_counts()

constitute         6
are                1
are disgrace to    1
Name: verb, dtype: int64

In [24]:
ieDF[ieDF['subject'] == 'Pericles']['verb'].value_counts()

sets ?? in    2
says in       2
sets          1
says to       1
Name: verb, dtype: int64

In [25]:
ieDF[ieDF['subject'] == 'Pericles']['object'].value_counts()

special relief                1
his Athenians                 1
funeral oration               1
celebrated funeral oration    1
relief                        1
??                            1
Name: object, dtype: int64

<span style="color:blue"> It's interesting that in Nietzsche's work the concept truth is related to tamed/domestic animals. It might implicate his opinion on truth: there is not truth, but discipline.